### Imports

In [18]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
import re
import unicodedata

### Declaración funciones

In [19]:
# Función build_model usada en los notebooks
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size,
                        output_dim=embedding_dim,
                        ))
    model.add(LSTM(rnn_units,
                    return_sequences=True,
                    stateful=True,
                    recurrent_initializer='glorot_uniform'))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))

    model.add(Dense(vocab_size))
    return model

# Función generate_text adaptada para recibir los parámetros necesarios
def generate_text(model, start_string, char2idx, idx2char):
    num_generate = 250
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 0.3 
    for layer in model.layers:
        if hasattr(layer, "reset_states"):
            layer.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return (start_string + ''.join(text_generated))

# Función filtrar_texto para "limpiar" el texto antes de pasarlo a otro modelo, para que no haya un conflicto de vo
def filtrar_texto(texto, vocab):
    texto_filtrado = ''.join(c for c in texto if c in vocab)
    return texto_filtrado if texto_filtrado.strip() else "Que te parece?"

# FUnción extraer_ultima_frase para extraer lo ultimo que dijo cada modelo antes de que el siguiente conteste
def extraer_ultima_frase(texto):
    frases = re.split(r'[,.]', texto.strip())
    if len(frases) == 0:
        return texto
    else:
        return frases[-1].strip() + '?'

### (Re)construcción de los modelos
Definiciones para utilizarse en la generación de texto

In [20]:
# Vocabs sacados del print(vocab) del notebook correspondiente a cada modelo
vocab_celestina = ['\n', ' ', '!', '"', '#', '&', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'X', 'Y', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '{', '}', '~', '¡', '¿', 'Ü', 'á', 'ç', 'é', 'í', 'ñ', 'ó', 'ú']
char2idx_celestina = {u:i for i, u in enumerate(vocab_celestina)}
idx2char_celestina = np.array(vocab_celestina)
embedding_dim_celestina = 256  # o el que usaste
rnn_units_celestina = 1024

vocab_lazarillo = ['\n', '\r', ' ', '!', '"', '(', ')', ',', '-', '.', '1', '4', '5', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '{', '}', '¡', '«', '»', '¿', 'É', 'á', 'é', 'ê', 'í', 'ñ', 'ó', 'ú', 'ü'] 
char2idx_lazarillo = {u:i for i, u in enumerate(vocab_lazarillo)}
idx2char_lazarillo = np.array(vocab_lazarillo)
embedding_dim_lazarillo = 256
rnn_units_lazarillo = 1024

vocab_quijote = ['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '¡', '«', '»', '¿', 'Á', 'É', 'Í', 'Ó', 'Ú', 'à', 'á', 'é', 'í', 'ï', 'ñ', 'ó', 'ù', 'ú', 'ü'] 
char2idx_quijote = {u:i for i, u in enumerate(vocab_quijote)}
idx2char_quijote = np.array(vocab_quijote)
embedding_dim_quijote = 256
rnn_units_quijote = 1024

# Construcción y carga pesos para cada modelo
model_celestina = build_model(len(vocab_celestina), embedding_dim_celestina, rnn_units_celestina, batch_size=1)
model_celestina.build(tf.TensorShape([1, None]))
model_celestina.load_weights('model_celestina_100_2025.keras')

model_lazarillo = build_model(len(vocab_lazarillo), embedding_dim_lazarillo, rnn_units_lazarillo, batch_size=1)
model_lazarillo.build(tf.TensorShape([1, None]))
model_lazarillo.load_weights('model_lazarillo_100_2025.keras')

model_quijote = build_model(len(vocab_quijote), embedding_dim_quijote, rnn_units_quijote, batch_size=1)
model_quijote.build(tf.TensorShape([1, None]))
model_quijote.load_weights('model_quijote_100_2025.keras')


### Diálogo
A cada modelo, salvo la primera vez que se utiliza el texto inicial, se le da lo último que dijo el modelo anterior. Este fragmento se usa como start_string, y el modelo se lo cuestiona, para hacer que fluya el diálogo.

**Ejemplo**:

Si Celestina dice: *"Baxa a él y dile que se pare."*

Lazarillo le responde: *"Baxa a él y dile que se pare? sin comer bocado, ni hablaba palabra..."*

In [21]:
# Texto inicial para iniciar el diálogo
texto = "Melibea, hija, ¿qué nueva me traes que vienes tan turbada?"

# Número de turnos (cada modelo genera en secuencia)
turnos = 5

for i in range(turnos):
    print(f"Turno {i+1} {'='*200}")

    # responde celestina
    respuesta_celestina = generate_text(model_celestina, texto, char2idx_celestina, idx2char_celestina)
    print("Celestina: ", respuesta_celestina)

    # se prepara lazarillo 
    texto = extraer_ultima_frase(respuesta_celestina)
    texto = filtrar_texto(texto, vocab_lazarillo)

    # responde lazarillo
    respuesta_lazarillo = generate_text(model_lazarillo, texto, char2idx_lazarillo, idx2char_lazarillo)
    print("Lazarillo: ", respuesta_lazarillo)

    # se prepara quijote
    texto = extraer_ultima_frase(respuesta_lazarillo)
    texto = filtrar_texto(texto, vocab_quijote)

    # responde quijote
    respuesta_quijote = generate_text(model_quijote, texto, char2idx_quijote, idx2char_quijote)
    print("Quijote: ", respuesta_quijote)

    # se prepara celestina
    texto = extraer_ultima_frase(respuesta_quijote)
    texto = filtrar_texto(texto, vocab_celestina)

    # print("="*300)

Turno 1 ========================================================================================================================================================================================================
Celestina:  Melibea, hija, ¿qué nueva me traes que vienes tan turbada?
Espantados traygo estas formas de pesces, ino !
kesí por me viuo con la Remano de
vosotros, no sé si orejas mias, que jamás propento, en que con
acordado cresciendo de vosotros sonos y acostados y lenguajes. ¿ Que me entendo sé
qué se ha vestido su 
Lazarillo:  ¿ Que me entendo sé
qué se ha vestido su?"

Y viejasastos mejor que un galgo suyo lo hiciera.

Jue aktado la negra que llaman Oes{", las cualendar mi amo y flegamerzüe el lasa, y no la cuieda la üencíaQÉ Él mes!


"Éste -decía yo-»» los por ko sin tienza ya la»z; y aun, as 15

Respon
Quijote:  as 15

Respon?

   Parte, es un caballero andante como un pasado, aunque no puede ser en
creerla con quien acompañarse; pues si eso es así, como es uso y costumbre

Aunque técnicamente funciona, el resultado es lingüísticamente caótico y la coherencia se degrada rápidamente. Poniendo a charlar a los modelos, se puede ver que no hay coherencia en lo que dicen o en lo que se contestan, pero tampoco se puede esperar coherencia narrativa entre tres modelos entrenados por separado con datasets de estilo muy diferente. 